In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
def input(dataset):
    return dataset.images, dataset.labels.astype(np.int32)

In [3]:
# Specify feature
feature_columns = [tf.feature_column.numeric_column('x', shape=[28, 28])]

In [ ]:
tf.estimator.DNNClassifier?

In [4]:
# Build 2 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=10,
    dropout=0.1,
    model_dir="./mnist_model",
    warm_start_from="./mnist_model"
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './mnist_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9cced03eb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': input(mnist.train)[0]},
    y=input(mnist.train)[1],
    num_epochs=None,
    batch_size=128,
    shuffle=True
)

In [ ]:
train_input_fn??

In [ ]:
classifier.train(input_fn=train_input_fn, steps=10000)

In [ ]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': input(mnist.test)[0]},
    y=input(mnist.test)[1],
    num_epochs=1,
    shuffle=False,
    batch_size =128
)

In [ ]:
# Evaluate accuracy
accuracy= classifier.evaluate(input_fn=test_input_fn)
print(accuracy)
#print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

#### Prepare to save a model expecting single `tf.string` tensor of serialized tf.Example

In [5]:
feature_spec = tf.feature_column.make_parse_example_spec(feature_columns)
export_input_receiver = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

#### Prepare to save a model expecting single tensor of `tf.float32` type

In [9]:
def export_input_receiver():
    inputs = {'x':tf.placeholder(dtype=tf.float32,shape=[None,28,28])}
    return tf.estimator.export.ServingInputReceiver(inputs,inputs)

In [10]:
classifier.export_savedmodel(".", serving_input_receiver_fn=export_input_receiver, as_text=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'x': <tf.Tensor 'Placeholder:0' shape=(?, 28, 28) dtype=float32>}
INFO:tensorflow:'classification' : Classification input must be a single string Tensor; got {'x': <tf.Tensor 'Placeholder:0' shape=(?, 28, 28) dtype=float32>}
INFO:tensorflow:Restoring parameters from ./mnist_model/model.ckpt-10000
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:A

b'./1544575913'

#### Try inference of two test images  against the saved model using `saved_model_cli`

In [25]:
from tensorflow.python.tools import saved_model_cli
test_images = np.array([mnist.test.images[i].reshape(28,28) for i in np.arange(2)])
np.save('test_images.npy',test_images)

In [27]:
!saved_model_cli run --dir ./1544575913 \
 --tag_set serve --signature_def predict \
 --inputs x=test_images.npy

2018-12-13 19:24:32.546990: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Result for output key class_ids:
[[7]
 [2]]
Result for output key classes:
[[b'7']
 [b'2']]
Result for output key logits:
[[ -3.2827048   -2.9382994    0.18175894   0.9892825   -5.3888254
   -5.200914   -11.60583      9.1448555   -0.80063385  -1.230514  ]
 [ -1.4085116   -0.54034853   9.427839     0.60019517 -11.020476
   -4.998603    -3.0489752   -6.528724     0.73463684 -10.855088  ]]
Result for output key probabilities:
[[4.0046043e-06 5.6511039e-06 1.2798447e-04 2.8698577e-04 4.8739747e-07
  5.8815527e-07 9.7246056e-10 9.9949515e-01 4.7919220e-05 3.1175598e-05]
 [1.9663690e-05 4.6849382e-05 9.9961472e-01 1.4656673e-04 1.3159591e-09
  5.4263580e-07 3.8125984e-06 1.1748569e-07 1.6765749e-04 1.5526327e-09]]
